In [1]:
import os
import pandas as pd
from datetime import datetime

c:\Users\jimen\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
base_path = r"C:\Users\jimen\Desktop\Marsh Mclennan\2024"
data_frames = []

In [5]:
# Recorrer todas las carpetas (año, mes, día)
for root, dirs, files in os.walk(base_path):
    # Extraer la fecha del nombre de la carpeta (formato esperado: dd-mm-yy)
    path_parts = root.split(os.sep)
    if len(path_parts) >= 3:  # Nos aseguramos de que esté en una carpeta de día dentro de mes
        try:
            date_str = path_parts[-1]  # Parte final de la ruta es el día
            date = datetime.strptime(date_str, "%d-%m-%y").date()
        except ValueError:
            continue  # Si no se puede convertir, continuar con la siguiente carpeta

        # Iterar sobre los archivos en la carpeta
        for file in files:
            file_path = os.path.join(root, file)
            # Leer archivos CSV o Excel
            try:
                if file.endswith(".csv"):
                    df = pd.read_csv(file_path, on_bad_lines='skip', delimiter=';', engine='python')
                elif file.endswith(".xlsx"):
                    try:
                        df = pd.read_excel(file_path, engine='openpyxl')
                    except ImportError:
                        print(f"Error: se requiere la versión '3.1.0' o superior de 'openpyxl' para leer {file_path}")
                        continue
                else:
                    continue

                # Limpiar columnas sin nombre (generalmente índice)
                df = df.loc[:, ~df.columns.str.contains('^Unnamed')]  # Eliminar columnas "Unnamed"

                # Renombrar columnas si es necesario
                rename_mapping = {
                    "Client_Name": "Client",
                    "País": "Country",
                    "Moneda": "Currency",
                    "Transacción": "Transaction"
                }
                df.rename(columns=rename_mapping, inplace=True)

                # Validar si las columnas esperadas existen
                expected_columns = ["Client", "Country", "Currency", "Transaction"]
                missing_columns = [col for col in expected_columns if col not in df.columns]
                if missing_columns:
                    print(f"Error: faltan columnas esperadas en el archivo {file_path}: {missing_columns}")
                    continue

                # Homogeneizar la columna 'Transaction'
                df['Transaction'] = df['Transaction'].astype(str)
                df['Transaction'] = df['Transaction'].str.replace(',', '.').str.replace(' ', '')  # Reemplazar ',' por '.' y eliminar espacios
                df['Transaction'] = pd.to_numeric(df['Transaction'], errors='coerce')  # Convertir a numérico

                # Redondear a dos decimales (opcional)
                df['Transaction'] = df['Transaction'].round(2)

                # Añadir la columna de fecha
                df["date"] = date

                # Asegurarse de que las columnas estén en el orden correcto
                expected_columns.append("date")
                df = df[expected_columns]

                # Agregar el DataFrame a la lista
                data_frames.append(df)

            except pd.errors.ParserError as e:
                print(f"Error al tokenizar el archivo {file_path}: {e}")
            except PermissionError:
                print(f"Error de permisos al intentar leer el archivo {file_path}")
            except Exception as e:
                print(f"Error al leer el archivo {file_path}: {e}")

In [6]:
# Concatenar todos los DataFrames
if data_frames:
    final_df = pd.concat(data_frames, ignore_index=True)
    
    # Guardar el DataFrame final a un archivo CSV
    # Cambiar la ruta de salida del archivo consolidado
    output_path = r"C:\Users\jimen\Desktop\Marsh Mclennan\Años\consolidated_data_2024.csv"
    final_df.to_csv(output_path, index=False)
    print(f"Archivo consolidado guardado en: {output_path}")
else:
    print("No se encontraron datos para consolidar.")

Archivo consolidado guardado en: C:\Users\jimen\Desktop\Marsh Mclennan\Años\consolidated_data_2024.csv
